# PyCity Schools Analysis
- Your analysis here


In [1]:
# Dependencies
import pandas as pd
from pathlib import Path

# Path to files
school_csv = Path("Resources/schools_complete.csv")
student_csv = Path("Resources/students_complete.csv")

# Read in CSV data to pandas DataFrames
school_data_unq = pd.read_csv(school_csv)
student_data_unq = pd.read_csv(student_csv)

# Merge the data  
school_merged_data = pd.merge(student_data_unq, school_data_unq, how="left", on=["school_name", "school_name"])
school_merged_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Count schools in the district
total_schools = len(school_merged_data['school_name'].unique())
total_schools

15

In [3]:
# Total students
total_students = len(school_merged_data['Student ID'])
total_students

39170

In [4]:
# Total budget
total_budget = school_data_unq['budget'].sum()
total_budget

24649428

In [5]:
# Average math score
avg_math_score = school_merged_data['math_score'].mean()
avg_math_score

78.98537145774827

In [6]:
# Average reading score
avg_read_score = school_merged_data['reading_score'].mean()
avg_read_score

81.87784018381414

In [7]:
# % passing math (the percentage of students who passed math)
students_pass_math = school_merged_data[(school_merged_data["math_score"] >= 70)].count()["student_name"]
percent_pass_math = students_pass_math / float(total_students) * 100
percent_pass_math

74.9808526933878

In [8]:
# % passing reading (the percentage of students who passed reading)
students_pass_read = school_merged_data[(school_merged_data["reading_score"] >= 70)].count()["student_name"]
percent_pass_read = students_pass_read / float(total_students) * 100
percent_pass_read

85.80546336482001

In [9]:
# % overall passing (the percentage of students who passed math AND reading)
students_pass_math_reading = school_merged_data[
    (school_merged_data["math_score"] >= 70) & (school_merged_data["reading_score"] >= 70)
].count()["student_name"]
overall_pass_rate = students_pass_math_reading /  float(total_students) * 100
overall_pass_rate

65.17232575950983

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary_df = pd.DataFrame({'Total Schools': [total_schools],
                                  'Total Students': [total_students],
                                   'Total Budget':[total_budget],
                                   'Average Math Score': [avg_math_score],
                                   'Average Reading Score':[avg_read_score],
                                   '% Passing Math':[percent_pass_math],
                                   '% Passing Reading':[percent_pass_read],
                                   '% Overall Passing':[overall_pass_rate]})

# Format
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

# Display
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [11]:
# select school types
school_types = pd.DataFrame(school_data_unq[['school_name','type']])

In [12]:
# Total students (per school)
students_per_school = pd.DataFrame(school_merged_data.groupby('school_name')['Student ID'].count())
students_per_school

,Student ID
school_name,
Bailey High School,4976
Cabrera High School,1858
Figueroa High School,2949
Ford High School,2739
Griffin High School,1468
Hernandez High School,4635
Holden High School,427
Huang High School,2917
Johnson High School,4761


In [33]:
# Total school budget and per capita spending (per school)
budget_per_school = pd.DataFrame(school_data_unq[['school_name','budget']])
per_school_capita = pd.DataFrame({'school_name': school_data_unq['school_name'],
                                  'Per Student Budget': school_data_unq['budget'] / school_data_unq['size']})


In [14]:
# Average test scores (per school)
avg_math_per_school = pd.DataFrame(school_merged_data.groupby('school_name')['math_score'].mean())
avg_read_per_school = pd.DataFrame(school_merged_data.groupby('school_name')['reading_score'].mean())

In [15]:
# #students w/ math >= 70 (per school)
stu_pass_math_per = pd.DataFrame(school_merged_data[school_merged_data["math_score"] >= 70].groupby('school_name')['Student ID'].count())

In [16]:
# #students w/ reading >= 70 (per school) 
stu_pass_read_per = pd.DataFrame(school_merged_data[school_merged_data["reading_score"] >= 70].groupby('school_name')['Student ID'].count())

In [17]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_pass_math_read = pd.DataFrame(school_merged_data[
    (school_merged_data["reading_score"] >= 70) & (school_merged_data["math_score"] >= 70)])
students_pass_math_read = pd.DataFrame(students_pass_math_read.groupby(["school_name"]).size())

In [39]:
# Use the provided code to calculate the passing rates
perc_pass_math_per = pd.DataFrame(stu_pass_math_per['Student ID'] / students_per_school['Student ID'] * 100)
perc_pass_read_per = pd.DataFrame(stu_pass_read_per['Student ID'] / students_per_school['Student ID'] * 100)
overall_pass_rate = pd.DataFrame(students_pass_math_read[0] / students_per_school['Student ID'] * 100)

In [42]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.merge(school_types,students_per_school, on="school_name",how='outer')
per_school_summary = pd.merge(per_school_summary, budget_per_school,on="school_name",how='outer')
per_school_summary = pd.merge(per_school_summary, per_school_capita,on="school_name",how='outer')
per_school_summary = pd.merge(per_school_summary, avg_math_per_school,on="school_name",how='outer')
per_school_summary = pd.merge(per_school_summary, avg_read_per_school,on="school_name",how='outer')
per_school_summary = pd.merge(per_school_summary, perc_pass_math_per,on="school_name",how='outer')
per_school_summary = pd.merge(per_school_summary, perc_pass_read_per,on="school_name",how='outer')
per_school_summary = pd.merge(per_school_summary, overall_pass_rate,on="school_name",how='outer')
# Rename columns
per_school_summary.columns = ['School Name', 'School Type', 'Total Students', 'Total School Budget','Per Student Budget','Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']
# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
2,Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


## Highest-Performing Schools (by % Overall Passing)

In [46]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
high_perf_schools = per_school_summary.sort_values('% Overall Passing',ascending=False)
high_perf_schools.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
14,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Lowest-Performing Schools (by % Overall Passing)

## Math Scores by Grade

## Reading Scores by Grade

## Scores by School Spending

## Scores by School Size

## Scores by School Type